In [2]:
import pandas as pd

# Load the dataset
file_path = "data/CeraVe_reddit_full_data_V2.csv"
df = pd.read_csv(file_path)

# Display the original number of records
original_count = len(df)

# Select relevant columns
columns_to_keep = ["title", "text", "upvotes", "comments", "timestamp", "url", "subreddit", "keyword"]
df = df[columns_to_keep]

# Remove missing values
df.dropna(subset=["title"], inplace=True)

# Drop duplicates
df.drop_duplicates(inplace=True)

# Convert to string, avoid issues with NaN values
df["comments"] = df["comments"].astype(str).fillna("")
df["text"] = df["text"].astype(str).fillna("")

# Timestamp
def convert_timestamp(value):
    try:
        if isinstance(value, (int, float)) and value > 1000000000:  # Unix timestamp check
            return pd.to_datetime(value, unit='s')
        else:
            return pd.to_datetime(value, errors='coerce')  # Convert or assign NaT
    except Exception:
        return pd.NaT

df["timestamp"] = df["timestamp"].apply(convert_timestamp)

# Define machine-generated content patterns
bot_keywords = ["bot", "automoderator"]

# Function to check if text contains bot-related keywords
def is_machine_generated(text):
    return any(keyword in text.lower() for keyword in bot_keywords)

# Remove machine-generated content from 'comments' and 'text' without deleting the entire row
df["comments"] = df["comments"].apply(lambda x: "" if is_machine_generated(x) else x)
df["text"] = df["text"].apply(lambda x: "" if is_machine_generated(x) else x)

df.reset_index(drop=True, inplace=True)

# Function to filter CeraVe-related comments
def filter_cerave_comments(comments):
    if pd.isna(comments) or comments.strip() == "":  
        return ""
    lines = comments.split(",")  
    filtered_lines = [line for line in lines if "cerave" in line.lower()]
    return "\n".join(filtered_lines) if filtered_lines else ""

# Apply filtering only if 'cerave' is NOT in the title
df["comments"] = df["comments"].map(filter_cerave_comments)

# Replace NaN values and reset index
df["comments"].replace(["nan", "NaN"], "", inplace=True)
df["text"].replace(["nan", "NaN"], "", inplace=True)
df.reset_index(drop=True, inplace=True)

# Save the cleaned dataset
cleaned_file_path = "data/CeraVe_reddit_cleaned_data_V2.csv"
df.to_csv(cleaned_file_path, index=False)

# Display statistics about removed data
cleaned_count = len(df)
removed_count = original_count - cleaned_count
removed_percentage = (removed_count / original_count) * 100

print(f"Cleaning complete! Remaining records: {cleaned_count}")
print(f"Removed {removed_count} records, accounting for {removed_percentage:.2f}% of the data.")
print(df[["title", "comments"]].head(10))


Cleaning complete! Remaining records: 7733
Removed 0 records, accounting for 0.00% of the data.
                                               title  \
0               [acne] A year after stopping Cerave    
1             [Review] CeraVe products love or hate?   
2  [B&A] A week ago I discovered this sub and dis...   
3  For all you CeraVe haters, what did CeraVe do ...   
4  [Product Question] Are Cerave products still w...   
5                      [Acne] is CeraVe really good?   
6  [PSA] I received this notice from Amazon that ...   
7  [Misc] CVS is not even trying to be subtle wit...   
8  [PSA] Being sold through the CeraVe Amazon sto...   
9  [acne] breakouts using Cerave moisturizer vs. ...   

                                            comments  
0                                                     
1                                                     
2                                                     
3                                                     
4           

In [3]:
import pandas as pd


# Load the cleaned dataset
file_path = "data/Cerave_reddit_cleaned_data_V2.csv"  
df = pd.read_csv(file_path)
# Check for missing values
missing_values = df.isnull().sum()

# Check for duplicate rows
duplicate_rows = df.duplicated().sum()

# Check data types
data_types = df.dtypes

# Check for empty strings after cleaning
empty_strings = (df == '').sum()

# Check for outliers in numeric columns
numeric_cols = df.select_dtypes(include=['number'])
outliers = numeric_cols.describe(percentiles=[0.25, 0.5, 0.75, 0.95, 0.99])

# Display results
print(" Missing Values Per Column:")
print(missing_values[missing_values > 0])

print("\n Duplicate Rows:", duplicate_rows)

print("\n Data Types:")
print(data_types)

print("\n Empty Strings Per Column:")
print(empty_strings[empty_strings > 0])

print("\n Outlier Analysis (Numeric Columns):")
print(outliers)


 Missing Values Per Column:
text        1987
comments    6618
dtype: int64

 Duplicate Rows: 0

 Data Types:
title        object
text         object
upvotes       int64
comments     object
timestamp    object
url          object
subreddit    object
keyword      object
dtype: object

 Empty Strings Per Column:
Series([], dtype: int64)

 Outlier Analysis (Numeric Columns):
            upvotes
count   7733.000000
mean     142.892668
std     1015.610470
min        0.000000
25%        1.000000
50%        3.000000
75%       15.000000
95%      550.000000
99%     3050.680000
max    63235.000000


In [8]:
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

file_path = "data/Cerave_reddit_cleaned_data_V2.csv"
df = pd.read_csv(file_path)

# Select relevant columns
columns_to_keep = ["title", "text", "upvotes", "comments", "timestamp", "url", "subreddit", "keyword"]
df = df[columns_to_keep]

# Remove missing values in title (title is required)
df.dropna(subset=["title"], inplace=True)

# Keep only one instance if there are duplicates in title and text
df.drop_duplicates(subset=["title", "text"], keep="first", inplace=True)

# Convert 'comments' and 'text' to string and fill NaN values
df["comments"] = df["comments"].astype(str).fillna("")
df["text"] = df["text"].astype(str).fillna("")

# Convert timestamp
df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")

# Remove bot-generated content from 'comments' and 'text' without deleting the row
bot_keywords = ["bot", "automoderator"]
def is_machine_generated(text):
    return any(keyword in text.lower() for keyword in bot_keywords)

df["comments"] = df["comments"].apply(lambda x: "" if is_machine_generated(x) else x)
df["text"] = df["text"].apply(lambda x: "" if is_machine_generated(x) else x)
# Remove URLs from text
df["text"] = df["text"].apply(lambda x: re.sub(r'http\S+|www\S+', '', x))

df.reset_index(drop=True, inplace=True)

# Combine 'title' and 'text' for NLP analysis**
df["full_text"] = df["title"].fillna("") + " " + df["text"].fillna("")

# clean text**
def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    tokens = word_tokenize(text)  # Tokenize words
    tokens = [word for word in tokens if word not in stopwords.words('english')]  # Remove stopwords
    return tokens


# Apply
df["tokens"] = df["full_text"].apply(clean_text)

# Save 
cleaned_file_path = "data/CeraVe_reddit_cleaned_data_V3.csv"
df.to_csv(cleaned_file_path, index=False)

# Display 
cleaned_count = len(df)
print(f"Cleaning complete! Remaining records: {cleaned_count}")
print(df[["title", "tokens"]].head(10))  # Show first 10 rows for verification


Cleaning complete! Remaining records: 4518
                                               title  \
0               [acne] A year after stopping Cerave    
1             [Review] CeraVe products love or hate?   
2  [B&A] A week ago I discovered this sub and dis...   
3  For all you CeraVe haters, what did CeraVe do ...   
4  [Product Question] Are Cerave products still w...   
5                      [Acne] is CeraVe really good?   
6  [PSA] I received this notice from Amazon that ...   
7  [Misc] CVS is not even trying to be subtle wit...   
8  [PSA] Being sold through the CeraVe Amazon sto...   
9  [acne] breakouts using Cerave moisturizer vs. ...   

                                              tokens  
0                [acne, year, stopping, cerave, nan]  
1        [review, cerave, products, love, hate, nan]  
2  [ba, week, ago, discovered, sub, discontinued,...  
3  [cerave, haters, cerave, skin, use, product, q...  
4  [product, question, cerave, products, still, d...  
5  [acne, 

# select time

In [5]:
import pandas as pd
from datetime import datetime

file_path = "data/CeraVe_reddit_cleaned_data_V3.csv"
df = pd.read_csv(file_path)

df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")  

current_year = datetime.now().year 
start_date = datetime(current_year - 5, 1, 1)  # 2020-01-01

df_filtered = df[df["timestamp"] >= start_date]

filtered_file_path = "data/CeraVe_reddit_filtered_last_5_years.csv"
df_filtered.to_csv(filtered_file_path, index=False)

print(f"Total records before filtering: {len(df)}")
print(f"Total records after filtering (last 5 years): {len(df_filtered)}")




Total records before filtering: 4518
Total records after filtering (last 5 years): 4006
